In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import json

In [7]:
ROOT = "/content/drive/MyDrive/Colab Notebooks/DM/논문참조"

In [8]:
df = pd.read_csv(f"{ROOT}/data/교내 주제선택 분반_위도경도 통합_null값 반영.csv")

In [10]:
dist = pd.read_csv(f"{ROOT}/data/분반_기관_distance.csv")
cost = pd.read_csv(f"{ROOT}/data/분반_기관_cost.csv")

In [11]:
new_order = list(cost.columns[2:12]) + list(cost.columns[:2]) + list(cost.columns[12:])

In [12]:
dist = dist.reindex(columns = new_order)
cost = cost.reindex(columns = new_order)

In [13]:
cos_sim = pd.read_csv(f"{ROOT}/data/contents_similarity.csv")

In [14]:
cost = cost.astype(int)

In [15]:
def find_matching2(idx):
    K = int(df.loc[idx]["예산(만원)"]/2)
    N = 358
    weight = [0] + list(cost.loc[idx])
    value = [0] + list(cos_sim.loc[idx])
    dp = [[[0] * (6) for _ in range(K+1)] for _ in range(N+1)]
    choiced = [[[[]] * (6) for _ in range(K+1)] for _ in range(N+1)]
    d = [0] + list(dist.loc[idx])
    
    for i in range(1,N+1):
        for j in range(1,K+1):
            for k in range(1,6):
                if d[i] > 60:
                    dp[i][j][k] = dp[i-1][j][k]
                    choiced[i][j][k] = choiced[i-1][j][k]
                    continue
                
                if j >= weight[i] and dp[i-1][j][k] < dp[i-1][j-weight[i]][k-1] + value[i]:
                        dp[i][j][k] = dp[i-1][j-weight[i]][k-1] + value[i]
                        choiced[i][j][k] = choiced[i-1][j-weight[i]][k-1] + [i]
                else:
                    dp[i][j][k] = dp[i-1][j][k]
                    choiced[i][j][k] = choiced[i-1][j][k]
    
    for key, values in match2.items():
        tmp = []
        for v in values:
            if v < 11:
                tmp.append(v + 2)
            elif v < 13:
                tmp.append(v - 10)
            else:
                tmp.append(v)
        tmp = [i-1 for i in tmp]        
        match2[key] = tmp
    
    for i in choiced[-1][-1][::-1]:
        if(len(i) != 0):
            match2[idx] = i
            break

In [ ]:
#match2 = {}
#for i in tqdm(range(1366)):
#    find_matching2(i)

In [19]:
program = pd.read_csv(f"{ROOT}/data/프로그램_통합.csv", encoding = 'cp949')
학교 = []
체험처 = []
학교_위도=[]
학교_경도=[]
체험처_위도=[]
체험처_경도=[]

for key, values in match2.items():
    #print(key)
    try:
        for i in values:
            학교.append(df['학교명'][int(key)])
            체험처.append(program['제목'][i])
            #print(df['학교명'][int(key)],":", program['제목'][i])
            학교_위도.append(df['위도'][int(key)])
            학교_경도.append(df['경도'][int(key)])
            체험처_위도.append(program['위도'][i])
            체험처_경도.append(program['경도'][i])  
    except:
        print(key, values)
matched = pd.DataFrame({'학교': 학교, '체험처': 체험처, '학교_위도':학교_위도, '학교_경도':학교_경도, '체험처_위도':체험처_위도, '체험처_경도':체험처_경도})
#matched.columns = ['학교명', '체험처']
matched

,학교,체험처,학교_위도,학교_경도,체험처_위도,체험처_경도
0,경북체육중학교,과학개념 완성 실험과학,35.812226,128.738053,36.428149,128.170728
1,경북체육중학교,과학수사 KCSI / 과학수사대는 무슨 일을 할까?,35.812226,128.738053,36.807056,128.629346
2,경북체육중학교,문화예술교육기반 메이커교육,35.812226,128.738053,35.982923,128.383916
3,경북체육중학교,특수분장사 체험,35.812226,128.738053,36.807056,128.629346
4,경북체육중학교,화랑도 체험활동,35.812226,128.738053,35.860172,129.189557
5,경북체육중학교,"김동리, 박목월 선생님 일대기(시인, 소설가와의 만남, 방송작가, 드라마 영화작가 ...",35.812226,128.738053,35.785536,129.319485
6,경북체육중학교,뭐야! 와인이 예술이라고!? 와인에 숨어있는 예술 찾기,35.812226,128.738053,35.923620,128.896450
7,경북체육중학교,수상스키 타기,35.812226,128.738053,35.956530,128.487850
8,경북체육중학교,예술로 마음을 도닥인다 : 도닥도닥 사회적 협동조합,35.812226,128.738053,35.825122,128.741329
9,경북체육중학교,화랑도 체험활동,35.812226,128.738053,35.860172,129.189557


In [20]:
matched.to_csv(f"{ROOT}/data/IP_.csv", index = False)